In [1]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import gym
import gym_Aircraft

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    
    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, n_latent_var):
        super(ActorCritic, self).__init__()

        # actor
        self.action_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.ReLU(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.ReLU(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.ReLU(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.ReLU(),
                nn.Linear(n_latent_var, action_dim),
                nn.Softmax(dim=-1)
                )
        
        # critic
        self.value_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.ReLU(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.ReLU(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.ReLU(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.ReLU(),
                nn.Linear(n_latent_var, 1)
                )
        
    def forward(self):
        raise NotImplementedError
        
    def act(self, state, memory):
        state = torch.from_numpy(state).float().to(device) 
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        
        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(dist.log_prob(action))
        
        return action.item()
    
    def evaluate(self, state, action):
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        
        state_value = self.value_layer(state)
        
        return action_logprobs, torch.squeeze(state_value), dist_entropy
        
class PPO:
    def __init__(self, state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.policy = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()
    
    def update(self, memory):   
        # Monte Carlo estimate of state rewards:
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        
        # Normalizing the rewards:
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)
        
        # convert list to tensor
        old_states = torch.stack(memory.states).to(device).detach()
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.stack(memory.logprobs).to(device).detach()
        
        # Optimize policy for K epochs:
        for _ in range(self.K_epochs):
            # Evaluating old actions and values :
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
            
            # Finding the ratio (pi_theta / pi_theta__old):
            ratios = torch.exp(logprobs - old_logprobs.detach())
                
            # Finding Surrogate Loss:
            advantages = rewards - state_values.detach()
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
        
        # Copy new weights into old policy:
        self.policy_old.load_state_dict(self.policy.state_dict())


In [2]:
# def main():
############## Hyperparameters ##############
env_name = "acav-v0"
# creating environment
env = gym.make(env_name)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
render = False
solved_reward = 100         # stop training if avg_reward > solved_reward
log_interval = 20           # print avg reward in the interval
max_episodes = 50000        # max training episodes
max_timesteps = 300         # max timesteps in one episode
n_latent_var = 64           # number of variables in hidden layer
update_timestep = 2000      # update policy every n timesteps
lr = 0.002
betas = (0.9, 0.999)
gamma = 0.99                # discount factor
K_epochs = 4                # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
random_seed = None
#############################################

if random_seed:
    torch.manual_seed(random_seed)
    env.seed(random_seed)

memory = Memory()
ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)

# logging variables
running_reward = 0
avg_length = 0
timestep = 0

In [3]:
# training loop
for i_episode in range(1, max_episodes+1):
    state = env.reset()
    for t in range(max_timesteps):
        timestep += 1

        # Running policy_old:
        action = ppo.policy_old.act(state, memory)
        state, reward, done, _ = env.step(action)

        # Saving reward and is_terminal:
        memory.rewards.append(reward)
        memory.is_terminals.append(done)

        # update if its time
        if timestep % update_timestep == 0:
            ppo.update(memory)
            memory.clear_memory()
            timestep = 0

        running_reward += reward
        if render:
            env.render()
        if done:
            break

    avg_length += t

    # stop training if avg_reward > solved_reward
    if running_reward > (log_interval*solved_reward):
        print("########## Solved! ##########")
        torch.save(ppo.policy.state_dict(), './PPO_{}.pth'.format(env_name))
        break

    # logging
    if i_episode % log_interval == 0:
        avg_length = int(avg_length/log_interval)
        running_reward = int((running_reward/log_interval))

        print('Episode {} \t avg length: {} \t reward: {}'.format(i_episode, avg_length, running_reward))
        running_reward = 0
        avg_length = 0

Episode 20 	 avg length: 98 	 reward: -15
Episode 40 	 avg length: 98 	 reward: -70
Episode 60 	 avg length: 98 	 reward: -65
Episode 80 	 avg length: 98 	 reward: -95
Episode 100 	 avg length: 98 	 reward: -65
Episode 120 	 avg length: 98 	 reward: -60
Episode 140 	 avg length: 98 	 reward: -65
Episode 160 	 avg length: 98 	 reward: -75
Episode 180 	 avg length: 98 	 reward: -60
Episode 200 	 avg length: 98 	 reward: -75
Episode 220 	 avg length: 98 	 reward: -75
Episode 240 	 avg length: 98 	 reward: -65
Episode 260 	 avg length: 98 	 reward: -75
Episode 280 	 avg length: 98 	 reward: -80
Episode 300 	 avg length: 98 	 reward: -75
Episode 320 	 avg length: 98 	 reward: -85
Episode 340 	 avg length: 98 	 reward: -65
Episode 360 	 avg length: 98 	 reward: -60
Episode 380 	 avg length: 98 	 reward: -65
Episode 400 	 avg length: 98 	 reward: -60
Episode 420 	 avg length: 99 	 reward: -70
Episode 440 	 avg length: 97 	 reward: -75
Episode 460 	 avg length: 98 	 reward: -60
Episode 480 	 a

Episode 3780 	 avg length: 99 	 reward: -55
Episode 3800 	 avg length: 98 	 reward: -95
Episode 3820 	 avg length: 98 	 reward: -65
Episode 3840 	 avg length: 98 	 reward: -60
Episode 3860 	 avg length: 98 	 reward: -70
Episode 3880 	 avg length: 98 	 reward: -70
Episode 3900 	 avg length: 98 	 reward: -65
Episode 3920 	 avg length: 98 	 reward: -90
Episode 3940 	 avg length: 98 	 reward: -75
Episode 3960 	 avg length: 98 	 reward: -95
Episode 3980 	 avg length: 98 	 reward: -65
Episode 4000 	 avg length: 98 	 reward: -60
Episode 4020 	 avg length: 98 	 reward: -70
Episode 4040 	 avg length: 98 	 reward: -60
Episode 4060 	 avg length: 98 	 reward: -70
Episode 4080 	 avg length: 98 	 reward: -70
Episode 4100 	 avg length: 97 	 reward: -80
Episode 4120 	 avg length: 97 	 reward: -75
Episode 4140 	 avg length: 98 	 reward: -60
Episode 4160 	 avg length: 98 	 reward: -75
Episode 4180 	 avg length: 98 	 reward: -65
Episode 4200 	 avg length: 98 	 reward: -70
Episode 4220 	 avg length: 98 	 

Episode 7520 	 avg length: 98 	 reward: -80
Episode 7540 	 avg length: 98 	 reward: -70
Episode 7560 	 avg length: 98 	 reward: -60
Episode 7580 	 avg length: 98 	 reward: -65
Episode 7600 	 avg length: 98 	 reward: -45
Episode 7620 	 avg length: 98 	 reward: -55
Episode 7640 	 avg length: 98 	 reward: -65
Episode 7660 	 avg length: 98 	 reward: -70
Episode 7680 	 avg length: 98 	 reward: -70
Episode 7700 	 avg length: 98 	 reward: -60
Episode 7720 	 avg length: 98 	 reward: -55
Episode 7740 	 avg length: 98 	 reward: -75
Episode 7760 	 avg length: 98 	 reward: -85
Episode 7780 	 avg length: 98 	 reward: -75
Episode 7800 	 avg length: 98 	 reward: -80
Episode 7820 	 avg length: 98 	 reward: -80
Episode 7840 	 avg length: 98 	 reward: -60
Episode 7860 	 avg length: 98 	 reward: -90
Episode 7880 	 avg length: 98 	 reward: -80
Episode 7900 	 avg length: 98 	 reward: -50
Episode 7920 	 avg length: 98 	 reward: -65
Episode 7940 	 avg length: 97 	 reward: -75
Episode 7960 	 avg length: 98 	 

Episode 11220 	 avg length: 98 	 reward: -60
Episode 11240 	 avg length: 98 	 reward: -75
Episode 11260 	 avg length: 98 	 reward: -75
Episode 11280 	 avg length: 98 	 reward: -65
Episode 11300 	 avg length: 98 	 reward: -85
Episode 11320 	 avg length: 98 	 reward: -65
Episode 11340 	 avg length: 98 	 reward: -75
Episode 11360 	 avg length: 98 	 reward: -70
Episode 11380 	 avg length: 98 	 reward: -70
Episode 11400 	 avg length: 98 	 reward: -90
Episode 11420 	 avg length: 98 	 reward: -70
Episode 11440 	 avg length: 98 	 reward: -75
Episode 11460 	 avg length: 98 	 reward: -65
Episode 11480 	 avg length: 98 	 reward: -75
Episode 11500 	 avg length: 98 	 reward: -75
Episode 11520 	 avg length: 98 	 reward: -65
Episode 11540 	 avg length: 98 	 reward: -60
Episode 11560 	 avg length: 98 	 reward: -65
Episode 11580 	 avg length: 98 	 reward: -55
Episode 11600 	 avg length: 98 	 reward: -85
Episode 11620 	 avg length: 98 	 reward: -55
Episode 11640 	 avg length: 98 	 reward: -70
Episode 11

Episode 14880 	 avg length: 98 	 reward: -65
Episode 14900 	 avg length: 98 	 reward: -70
Episode 14920 	 avg length: 98 	 reward: -65
Episode 14940 	 avg length: 98 	 reward: -55
Episode 14960 	 avg length: 98 	 reward: -70
Episode 14980 	 avg length: 98 	 reward: -50
Episode 15000 	 avg length: 98 	 reward: -50
Episode 15020 	 avg length: 98 	 reward: -65
Episode 15040 	 avg length: 98 	 reward: -60
Episode 15060 	 avg length: 98 	 reward: -50
Episode 15080 	 avg length: 98 	 reward: -65
Episode 15100 	 avg length: 98 	 reward: -50
Episode 15120 	 avg length: 98 	 reward: -55
Episode 15140 	 avg length: 98 	 reward: -60
Episode 15160 	 avg length: 98 	 reward: -80
Episode 15180 	 avg length: 98 	 reward: -50
Episode 15200 	 avg length: 98 	 reward: -40
Episode 15220 	 avg length: 98 	 reward: -60
Episode 15240 	 avg length: 98 	 reward: -85
Episode 15260 	 avg length: 98 	 reward: -65
Episode 15280 	 avg length: 98 	 reward: -55
Episode 15300 	 avg length: 98 	 reward: -70
Episode 15

...

Episode 33180 	 avg length: 97 	 reward: -35
Episode 33200 	 avg length: 98 	 reward: -45
Episode 33220 	 avg length: 98 	 reward: -55
Episode 33240 	 avg length: 97 	 reward: -60
Episode 33260 	 avg length: 97 	 reward: -50
Episode 33280 	 avg length: 98 	 reward: -50
Episode 33300 	 avg length: 98 	 reward: -60
Episode 33320 	 avg length: 98 	 reward: -60
Episode 33340 	 avg length: 97 	 reward: -60
Episode 33360 	 avg length: 97 	 reward: -60
Episode 33380 	 avg length: 98 	 reward: -50
Episode 33400 	 avg length: 97 	 reward: -65
Episode 33420 	 avg length: 98 	 reward: -55
Episode 33440 	 avg length: 98 	 reward: -70
Episode 33460 	 avg length: 98 	 reward: -75
Episode 33480 	 avg length: 97 	 reward: -45
Episode 33500 	 avg length: 97 	 reward: -40
Episode 33520 	 avg length: 98 	 reward: -55
Episode 33540 	 avg length: 97 	 reward: -60
Episode 33560 	 avg length: 98 	 reward: -50
Episode 33580 	 avg length: 98 	 reward: -75
Episode 33600 	 avg length: 98 	 reward: -50
Episode 33

Episode 36840 	 avg length: 97 	 reward: -35
Episode 36860 	 avg length: 98 	 reward: -55
Episode 36880 	 avg length: 98 	 reward: -60
Episode 36900 	 avg length: 97 	 reward: -60
Episode 36920 	 avg length: 98 	 reward: -35
Episode 36940 	 avg length: 98 	 reward: -45
Episode 36960 	 avg length: 97 	 reward: -60
Episode 36980 	 avg length: 98 	 reward: -65
Episode 37000 	 avg length: 98 	 reward: -80
Episode 37020 	 avg length: 98 	 reward: -45
Episode 37040 	 avg length: 97 	 reward: -40
Episode 37060 	 avg length: 97 	 reward: -50
Episode 37080 	 avg length: 98 	 reward: -65
Episode 37100 	 avg length: 98 	 reward: -40
Episode 37120 	 avg length: 98 	 reward: -65
Episode 37140 	 avg length: 98 	 reward: -60
Episode 37160 	 avg length: 98 	 reward: -65
Episode 37180 	 avg length: 98 	 reward: -75
Episode 37200 	 avg length: 98 	 reward: -50
Episode 37220 	 avg length: 98 	 reward: -70
Episode 37240 	 avg length: 97 	 reward: -70
Episode 37260 	 avg length: 98 	 reward: -65
Episode 37

Episode 40500 	 avg length: 98 	 reward: -40
Episode 40520 	 avg length: 98 	 reward: -55
Episode 40540 	 avg length: 98 	 reward: -60
Episode 40560 	 avg length: 97 	 reward: -65
Episode 40580 	 avg length: 98 	 reward: -30
Episode 40600 	 avg length: 98 	 reward: -75
Episode 40620 	 avg length: 98 	 reward: -60
Episode 40640 	 avg length: 97 	 reward: -45
Episode 40660 	 avg length: 98 	 reward: -75
Episode 40680 	 avg length: 98 	 reward: -55
Episode 40700 	 avg length: 98 	 reward: -45
Episode 40720 	 avg length: 98 	 reward: -60
Episode 40740 	 avg length: 98 	 reward: -60
Episode 40760 	 avg length: 98 	 reward: -40
Episode 40780 	 avg length: 98 	 reward: -45
Episode 40800 	 avg length: 98 	 reward: -65
Episode 40820 	 avg length: 98 	 reward: -50
Episode 40840 	 avg length: 98 	 reward: -40
Episode 40860 	 avg length: 98 	 reward: -65
Episode 40880 	 avg length: 98 	 reward: -50
Episode 40900 	 avg length: 98 	 reward: -45
Episode 40920 	 avg length: 98 	 reward: -50
Episode 40

Episode 44160 	 avg length: 97 	 reward: -20
Episode 44180 	 avg length: 98 	 reward: -10
Episode 44200 	 avg length: 97 	 reward: -10
Episode 44220 	 avg length: 98 	 reward: -20
Episode 44240 	 avg length: 98 	 reward: -25
Episode 44260 	 avg length: 98 	 reward: -25
Episode 44280 	 avg length: 98 	 reward: -30
Episode 44300 	 avg length: 98 	 reward: -10
Episode 44320 	 avg length: 98 	 reward: -25
Episode 44340 	 avg length: 97 	 reward: -35
Episode 44360 	 avg length: 97 	 reward: -10
Episode 44380 	 avg length: 97 	 reward: -5
Episode 44400 	 avg length: 98 	 reward: -5
Episode 44420 	 avg length: 98 	 reward: -15
Episode 44440 	 avg length: 98 	 reward: -40
Episode 44460 	 avg length: 98 	 reward: -20
Episode 44480 	 avg length: 98 	 reward: -20
Episode 44500 	 avg length: 98 	 reward: -35
Episode 44520 	 avg length: 98 	 reward: -40
Episode 44540 	 avg length: 97 	 reward: -20
Episode 44560 	 avg length: 98 	 reward: -5
Episode 44580 	 avg length: 98 	 reward: -25
Episode 44600

Episode 47860 	 avg length: 98 	 reward: -35
Episode 47880 	 avg length: 98 	 reward: -10
Episode 47900 	 avg length: 98 	 reward: -10
Episode 47920 	 avg length: 98 	 reward: -10
Episode 47940 	 avg length: 98 	 reward: -10
Episode 47960 	 avg length: 98 	 reward: -25
Episode 47980 	 avg length: 97 	 reward: -20
Episode 48000 	 avg length: 97 	 reward: -30
Episode 48020 	 avg length: 98 	 reward: -25
Episode 48040 	 avg length: 98 	 reward: -15
Episode 48060 	 avg length: 97 	 reward: -15
Episode 48080 	 avg length: 98 	 reward: -25
Episode 48100 	 avg length: 97 	 reward: -15
Episode 48120 	 avg length: 98 	 reward: -5
Episode 48140 	 avg length: 98 	 reward: -5
Episode 48160 	 avg length: 97 	 reward: -10
Episode 48180 	 avg length: 98 	 reward: -5
Episode 48200 	 avg length: 99 	 reward: -30
Episode 48220 	 avg length: 98 	 reward: -15
Episode 48240 	 avg length: 98 	 reward: -15
Episode 48260 	 avg length: 98 	 reward: -25
Episode 48280 	 avg length: 98 	 reward: -5
Episode 48300 

In [4]:
memory

In [5]:
# if __name__ == '__main__':
#     main()